In [ ]:
#| default_exp models.classifiers

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import torch, numpy as np
from typing import Optional
import torch.nn as nn
from dataclasses import dataclass
import torch.nn.functional as F

from xcai.core import store_attr
from xcai.losses import MultiTriplet
from xcai.models.modeling_utils import XCModelOutput

from transformers.utils.generic import ModelOutput
from transformers import DistilBertPreTrainedModel,DistilBertConfig

In [ ]:
import os,torch, pickle, numpy as np
from torch.utils.data import Dataset

from xcai.block import *
from xcai.basics import *
from xcai.models.PPP0XX import DBT010

from fastcore.utils import *

from safetensors import safe_open

comet_ml is installed but `COMET_API_KEY` is not set.


## Setup

In [ ]:
data_dir = '/home/scai/phd/aiz218323/Projects/XC_NLG/data'

block = XCBlock.from_cfg(data_dir, 'data_meta', transform_type='xcs', tokenizer='distilbert-base-uncased', 
                         sampling_features=[('lbl2data,cat2lbl2data',1), ('cat2data',1)])

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets/'
pkl_file = f'{pkl_dir}/processed/wikiseealsotitles_data-meta_distilbert-base-uncased_xcs.pkl'

In [ ]:
with open(pkl_file, 'wb') as file: pickle.dump(block, file)

In [ ]:
with open(pkl_file, 'rb') as file: block = pickle.load(file)

In [ ]:
args = XCLearningArguments(
    output_dir='/home/scai/phd/aiz218323/scratch/outputs/69-distillation-for-wikiseealso-1-4',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=10,
    save_strategy="steps",
    evaluation_strategy="steps",
    eval_steps=3000,
    save_steps=3000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    representation_search_type='BRUTEFORCE',
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,
    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=25,
    clustering_type='EXPO',
    minimum_cluster_size=2,
    maximum_cluster_size=1600,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    use_encoder_parallel=True,
    max_grad_norm=None,
    fp16=True,
    label_names=['lbl2data_idx', 'lbl2data_input_ids', 'lbl2data_attention_mask'],
    use_data_metadata_for_representation=True,
)


In [ ]:
model_output = '/home/scai/phd/aiz218323/scratch/outputs/67-ngame-ep-for-wikiseealso-with-input-concatenation-1-4'
output_dir = f"/home/scai/phd/aiz218323/scratch/outputs/{os.path.basename(model_output)}"
mname = f'{output_dir}/{os.path.basename(get_best_model(output_dir))}'

model = DBT010.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=800, tn_targ=5000, margin=0.3, tau=0.1, 
                               n_negatives=10, apply_softmax=True, use_encoder_parallel=True)

model_weight_file,model_weights = f'{mname}/model.safetensors',{}
with safe_open(model_weight_file, framework="pt") as file:
    for k in file.keys(): model_weights[k] = file.get_tensor(k)

model.load_state_dict(model_weights, strict=False)

Some weights of DBT010 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.dr_layer_norm.bias', 'encoder.dr_layer_norm.weight', 'encoder.dr_projector.bias', 'encoder.dr_projector.weight', 'encoder.dr_transform.bias', 'encoder.dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


_IncompatibleKeys(missing_keys=['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'distilbert.transformer.layer.0.ffn.lin2.bias', 'distilbert.transformer.layer.

In [ ]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [ ]:
learn = XCLearner(
    model=model, 
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
data_rep, lbl_rep = learn.get_data_and_lbl_representation(learn.train_dataset)

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/434 [00:00<?, ?it/s]

## `CLS001`

In [ ]:
#| export
class CLS001(DistilBertPreTrainedModel):

    def __init__(
        self, 
        config, 
        n_data:int, 
        n_lbl:int, 
        num_batch_labels:Optional[int]=None, 
        batch_size:Optional[int]=None,
        margin:Optional[float]=0.3,
        num_negatives:Optional[int]=5,
        tau:Optional[float]=0.1,
        apply_softmax:Optional[bool]=True,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        store_attr('n_data,n_lbl')
        self.data_repr = nn.Embedding(self.n_data, config.dim)
        self.lbl_repr = nn.Embedding(self.n_lbl, config.dim)
        self.lbl_embeddings = nn.Embedding(self.n_lbl, config.dim)

        self.rep_loss_fn = MultiTriplet(bsz=batch_size, tn_targ=num_batch_labels, margin=margin, n_negatives=num_negatives, 
                                        tau=tau, apply_softmax=apply_softmax, reduce='mean')

    def get_lbl_representation(self):
        return self.lbl_repr.weight

    def get_data_representation(self):
        return self.data_repr.weight

    def init_representation(self, data_repr:torch.Tensor, lbl_repr:torch.Tensor):
        self.data_repr.weight.data = data_repr
        self.lbl_repr.weight.data = lbl_repr

    def freeze_representation(self):
        self.data_repr.requires_grad_(False)
        self.lbl_repr.requires_grad_(False)

    def freeze_data_representation(self):
        self.data_repr.requires_grad_(False)

    def unfreeze_representation(self):
        self.data_repr.requires_grad_(True)
        self.lbl_repr.requires_grad_(True)

    def init_meta_embeddings(self):
        self.lbl_embeddings.weight.data = torch.zeros_like(self.lbl_embeddings.weight.data)

    def compute_loss(self, inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx):
        return self.rep_loss_fn(inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx)

    def forward(
        self,
        data_idx:torch.Tensor,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
    ):
        data_rep = self.data_repr(data_idx)

        loss = lbl2data_rep = None
        if lbl2data_idx is not None:
            lbl2data_rep = F.normalize(self.lbl_repr(lbl2data_idx) + self.lbl_embeddings(lbl2data_idx))

            loss = self.compute_loss(data_rep, lbl2data_rep,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)
            
        return XCModelOutput(
            loss=loss,
            data_repr=data_rep,
            lbl2data_repr=lbl2data_rep,
        )
        
        

### Example

In [ ]:
model = CLS001(DistilBertConfig(), n_data=block.train.dset.n_data, n_lbl=block.n_lbl, batch_size=100, num_batch_labels=5000, 
               margin=0.3, num_negatives=10, tau=0.1, apply_softmax=True)
model.init_representation(data_rep, lbl_rep)
model.freeze_representation()

In [ ]:
batch = next(iter(block.train.dl))
b = prepare_batch(model, batch, m_args=['lbl2data_data2ptr', 'lbl2data_idx', 'plbl2data_data2ptr', 'plbl2data_idx'])

In [ ]:
o = model(**b)

In [ ]:
o.loss

tensor(0.0303, grad_fn=<DivBackward0>)